In [1]:
import statsmodels.formula.api as smf
import fastreg as fr
from fastreg import I, R, C, C0

### Generate Data

In [2]:
models = ['linear', 'poisson', 'logit', 'negbin']
data = fr.dataset(N=1_000_000, K1=10, K2=100, models=models, seed=89320432)
data_wide = fr.dataset(N=1_000_000, K1=10, K2=10_000, models=models, seed=89320433)
data.head()

,x1,x2,yhat0,yhat,id1,id2,y0,y,Eb0,Eb,b0,b,Ep0,Ep,p0,p,nb0,nb
0,-1.429225,1.830295,0.769410,2.149410,H,68,1.791671,1.585510,0.683393,0.895614,1,1,2.158492,8.579792,4,10,1,5
1,0.687153,0.795373,0.783369,1.403369,C,42,0.295552,3.163529,0.686406,0.802718,1,1,2.188835,4.068886,1,2,1,0
2,0.764353,-1.060225,-0.306829,-0.036829,C,7,0.381637,-0.639362,0.423889,0.490794,1,1,0.735777,0.963841,0,1,2,0
3,-0.309887,-0.777701,-0.459587,0.800413,D,96,0.143218,1.139854,0.387084,0.690063,0,1,0.631545,2.226461,0,0,1,3
4,-0.616042,-0.752266,-0.536172,0.623828,H,46,-0.468395,1.462211,0.369078,0.651089,1,1,0.584983,1.866057,0,1,1,7


In [3]:
# for statsmodels runs
data1 = data.copy()
data1['id2'] = data1['id2'].astype(str)

### Normal OLS

In [4]:
%time fr.ols(y=R.y0, x=I+R.x1+R.x2, data=data)

CPU times: user 37.5 ms, sys: 90.2 ms, total: 128 ms
Wall time: 22.6 ms


y0,coeff,stderr,low95,high95,pvalue
I,0.101859,0.001000,0.099900,0.103818,0.0
x1,0.301647,0.001000,0.299688,0.303606,0.0
x2,0.599408,0.000999,0.597450,0.601366,0.0


In [5]:
%time smf.ols('y0 ~ 1 + x1 + x2', data=data).fit().params

CPU times: user 336 ms, sys: 1.25 s, total: 1.59 s
Wall time: 132 ms


Intercept    0.101859
x1           0.301647
x2           0.599408
dtype: float64

In [6]:
%time fr.ols(y=R.y, x=I+R.x1+R.x2+C.id1+C.id2, data=data)

CPU times: user 437 ms, sys: 672 ms, total: 1.11 s
Wall time: 327 ms


y,coeff,stderr,low95,high95,pvalue
I,0.112463,0.010413,0.092054,0.132872,0.0
x1,0.299557,0.001001,0.297595,0.301518,0.0
x2,0.601840,0.001000,0.599880,0.603800,0.0
id1=B,0.104734,0.004476,0.095962,0.113507,0.0
id1=C,0.201965,0.004474,0.193197,0.210734,0.0
...,...,...,...,...,...
id2=95,0.937461,0.014102,0.909821,0.965101,0.0
id2=96,0.956624,0.014109,0.928970,0.984278,0.0
id2=97,0.960951,0.014099,0.933318,0.988584,0.0
id2=98,0.993398,0.014132,0.965699,1.021097,0.0


In [7]:
%time smf.ols('y ~ 1 + x1 + x2 + id1 + id2', data=data1).fit().params

CPU times: user 1min 37s, sys: 19.9 s, total: 1min 57s
Wall time: 11 s


Intercept    0.112463
id1[T.B]     0.104734
id1[T.C]     0.201965
id1[T.D]     0.300212
id1[T.E]     0.403094
               ...   
id2[T.97]    0.960951
id2[T.98]    0.993398
id2[T.99]    0.988670
x1           0.299557
x2           0.601840
Length: 111, dtype: float64

### High Dimensional

In [8]:
%time fr.ols(y=R.y, x=I+R.x1+R.x2+C.id1+C.id2, data=data_wide)

CPU times: user 4.17 s, sys: 2.26 s, total: 6.43 s
Wall time: 4.55 s


y,coeff,stderr,low95,high95,pvalue
I,0.143877,0.103699,-0.059369,0.347123,1.653041e-01
x1,0.301630,0.001004,0.299662,0.303597,0.000000e+00
x2,0.599284,0.001006,0.597313,0.601255,0.000000e+00
id1=B,0.097642,0.004499,0.088824,0.106461,0.000000e+00
id1=C,0.189008,0.004491,0.180207,0.197810,0.000000e+00
...,...,...,...,...,...
id2=9995,1.045182,0.146196,0.758643,1.331722,8.730794e-13
id2=9996,1.029893,0.146587,0.742588,1.317198,2.128075e-12
id2=9997,0.892911,0.142021,0.614554,1.171267,3.233234e-10
id2=9998,0.961372,0.144705,0.677755,1.244989,3.059886e-11


In [9]:
%time fr.ols(y=R.y, x=I+R.x1+R.x2+C.id1, hdfe=C.id2, data=data_wide)

CPU times: user 375 ms, sys: 22.1 ms, total: 397 ms
Wall time: 359 ms


y,coeff,stderr,low95,high95,pvalue
I,0.143877,0.103699,-0.059369,0.347123,1.653041e-01
x1,0.301630,0.001004,0.299662,0.303597,0.000000e+00
x2,0.599284,0.001006,0.597313,0.601255,0.000000e+00
id1=B,0.097642,0.004499,0.088824,0.106461,0.000000e+00
id1=C,0.189008,0.004491,0.180207,0.197810,0.000000e+00
...,...,...,...,...,...
id2=9995,1.045182,0.146196,0.758643,1.331722,8.730794e-13
id2=9996,1.029893,0.146587,0.742588,1.317198,2.128075e-12
id2=9997,0.892911,0.142021,0.614554,1.171267,3.233234e-10
id2=9998,0.961372,0.144705,0.677755,1.244989,3.059886e-11


In [10]:
%time fr.ols(y=R.y, x=I+R.x1+R.x2+C.id1, absorb=C.id2, data=data_wide)

CPU times: user 800 ms, sys: 1.78 s, total: 2.58 s
Wall time: 355 ms


y,coeff,stderr,low95,high95,pvalue
I,0.605524,0.003035,0.599576,0.611471,0.0
x1,0.301630,0.000998,0.299674,0.303585,0.0
x2,0.599284,0.001014,0.597297,0.601270,0.0
id1=B,0.097642,0.004531,0.088762,0.106523,0.0
id1=C,0.189008,0.004545,0.180100,0.197917,0.0
id1=D,0.295160,0.004506,0.286328,0.303991,0.0
id1=E,0.402802,0.004510,0.393963,0.411641,0.0
id1=F,0.495409,0.004493,0.486602,0.504215,0.0
id1=G,0.596910,0.004467,0.588155,0.605665,0.0
id1=H,0.694326,0.004490,0.685526,0.703125,0.0


### Poisson

In [11]:
%time fr.poisson(y=R.p0, x=I+R.x1+R.x2, data=data)

[  0] ℓ=-0.68046, g=0.25030, Δβ=0.59787, Δℓ=inf, μR=0.33246, μC=nan
[  8] ℓ=-0.62217, g=0.00006, Δβ=0.00003, Δℓ=0.00000, μR=0.33603, μC=nan
CPU times: user 1.38 s, sys: 230 ms, total: 1.6 s
Wall time: 1.43 s


p0,coeff,stderr,low95,high95,pvalue
I,0.098944,0.001023,0.096939,0.100948,0.0
x1,0.305033,0.000849,0.303370,0.306697,0.0
x2,0.604124,0.000847,0.602464,0.605784,0.0


In [12]:
%time fr.poisson(y=R.p, x=I+R.x1+R.x2+C.id1+C.id2, data=data)

[  0] ℓ=1.94576, g=0.78521, Δβ=0.75252, Δℓ=inf, μR=0.44628, μC=0.32821
[ 40] ℓ=2.55464, g=0.00008, Δβ=0.00102, Δℓ=0.00000, μR=0.34047, μC=0.49258
[ 71] ℓ=2.55464, g=0.00008, Δβ=0.00010, Δℓ=0.00000, μR=0.33658, μC=0.49842
CPU times: user 13.7 s, sys: 2.74 s, total: 16.4 s
Wall time: 10.3 s


p,coeff,stderr,low95,high95,pvalue
I,0.104745,0.006941,0.091141,0.118348,0.0
x1,0.300882,0.000507,0.299887,0.301876,0.0
x2,0.604118,0.000505,0.603127,0.605108,0.0
id1=B,0.096785,0.002832,0.091236,0.102335,0.0
id1=C,0.197594,0.002770,0.192165,0.203023,0.0
...,...,...,...,...,...
id2=95,0.954749,0.007782,0.939497,0.970001,0.0
id2=96,0.937989,0.007811,0.922680,0.953298,0.0
id2=97,0.970691,0.007817,0.955370,0.986012,0.0
id2=98,1.002750,0.007773,0.987516,1.017984,0.0


### Logit

In [13]:
%time fr.logit(y=R.b0, x=I+R.x1+R.x2, data=data)

[  0] ℓ=-0.65131, g=0.03946, Δβ=0.60313, Δℓ=inf, μR=0.33769, μC=nan
[  7] ℓ=-0.64338, g=0.00007, Δβ=0.00003, Δℓ=0.00000, μR=0.34135, μC=nan
CPU times: user 1.06 s, sys: 137 ms, total: 1.2 s
Wall time: 849 ms


b0,coeff,stderr,low95,high95,pvalue
I,0.107386,0.002104,0.103263,0.111510,0.0
x1,0.309365,0.002146,0.305160,0.313571,0.0
x2,0.607311,0.002269,0.602863,0.611758,0.0


In [14]:
%time fr.logit(y=R.b, x=I+R.x1+R.x2+C.id1+C.id2, data=data)

[  0] ℓ=-0.56577, g=0.03554, Δβ=0.68609, Δℓ=inf, μR=0.43241, μC=0.33395
[ 40] ℓ=-0.54390, g=0.00004, Δβ=0.00161, Δℓ=0.00000, μR=0.36243, μC=0.45748
[ 80] ℓ=-0.54390, g=0.00004, Δβ=0.00043, Δℓ=0.00000, μR=0.35069, μC=0.47531
[120] ℓ=-0.54390, g=0.00004, Δβ=0.00012, Δℓ=0.00000, μR=0.34754, μC=0.48010
[125] ℓ=-0.54390, g=0.00004, Δβ=0.00010, Δℓ=0.00000, μR=0.34737, μC=0.48037
CPU times: user 22.6 s, sys: 2.68 s, total: 25.3 s
Wall time: 17 s


b,coeff,stderr,low95,high95,pvalue
I,0.146581,0.022841,0.101814,0.191348,1.385494e-10
x1,0.302469,0.002382,0.297801,0.307136,0.000000e+00
x2,0.593056,0.002508,0.588140,0.597971,0.000000e+00
id1=B,0.077370,0.009834,0.058095,0.096645,3.552714e-15
id1=C,0.204268,0.009909,0.184846,0.223690,0.000000e+00
...,...,...,...,...,...
id2=95,0.840746,0.033493,0.775102,0.906391,0.000000e+00
id2=96,0.956823,0.034054,0.890077,1.023568,0.000000e+00
id2=97,0.910034,0.033868,0.843655,0.976413,0.000000e+00
id2=98,0.911207,0.034025,0.844519,0.977895,0.000000e+00


### Ultra Wide

In [15]:
N = 5_000_000
df = pd.DataFrame({ 
    'x1': np.random.rand(N), 
    'x2': np.random.rand(N), 
    'id1': np.ceil(10*np.arange(N)/N+1e-7).astype(int),
    'id2': np.random.randint(1, 100_001, size=N)
})
df['y'] = 1 + 2*df['x1'] + 3*df['x2'] + np.log10(df['id1']) + np.log10(df['id2']) + np.random.randn(N)
print(df[['id1', 'id2']].nunique())

id1        10
id2    100000
dtype: int64


In [16]:
%time fr.ols(y=R.y, x=I+R.x1+R.x2+C.id1, hdfe=C.id2, data=df)

CPU times: user 2.04 s, sys: 600 ms, total: 2.64 s
Wall time: 2.02 s


y,coeff,stderr,low95,high95,pvalue
I,1.024619,0.164455,0.702294,1.346944,4.652241e-10
x1,2.000838,0.001565,1.997770,2.003906,0.000000e+00
x2,3.001286,0.001566,2.998217,3.004355,0.000000e+00
id1=2,0.298016,0.002021,0.294055,0.301977,0.000000e+00
id1=3,0.475738,0.002021,0.471777,0.479699,0.000000e+00
...,...,...,...,...,...
id2=99996,4.876849,0.210459,4.464357,5.289341,0.000000e+00
id2=99997,4.833998,0.213473,4.415598,5.252398,0.000000e+00
id2=99998,5.054997,0.209088,4.645192,5.464803,0.000000e+00
id2=99999,4.819549,0.212683,4.402699,5.236400,0.000000e+00
